In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rfcx-species-audio-detection/sample_submission.csv
/kaggle/input/rfcx-species-audio-detection/train_fp.csv
/kaggle/input/rfcx-species-audio-detection/train_tp.csv
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/18-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/08-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/13-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/11-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/02-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/07-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/27-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/23-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/04-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/30-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/01-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/

In [2]:
import csv
import librosa
from skimage.transform import resize
from PIL import Image

fft = 2048
hop = 512
# Less rounding errors this way
sr = 48000
length = 10 * sr

with open('/kaggle/input/rfcx-species-audio-detection/train_tp.csv') as f:
    reader = csv.reader(f)
    data = list(reader)
    
data

[['recording_id',
  'species_id',
  'songtype_id',
  't_min',
  'f_min',
  't_max',
  'f_max'],
 ['003bec244', '14', '1', '44.544', '2531.25', '45.1307', '5531.25'],
 ['006ab765f', '23', '1', '39.9615', '7235.16', '46.0452', '11283.4'],
 ['007f87ba2', '12', '1', '39.136', '562.5', '42.272', '3281.25'],
 ['0099c367b', '17', '4', '51.4206', '1464.26', '55.1996', '4565.04'],
 ['009b760e6', '10', '1', '50.0854', '947.461', '52.5293', '10852.7'],
 ['00b404881', '8', '1', '0.0747', '3750.0', '4.1973', '5531.25'],
 ['00d442df7', '0', '1', '19.3653', '5906.25', '20.16', '8250.0'],
 ['011f25080', '18', '1', '5.6853', '3187.5', '6.3787', '5062.5'],
 ['015113cad', '15', '1', '50.0533', '93.75', '53.3973', '1125.0'],
 ['0151b7d20', '1', '1', '46.032', '3843.75', '46.928', '5625.0'],
 ['01b41f92b', '6', '1', '44.24', '562.5', '46.384', '4406.25'],
 ['0201197ec', '10', '1', '43.3575', '947.461', '45.8014', '10852.7'],
 ['0209f7ab2', '7', '1', '50.7573', '4687.5', '53.8987', '11437.5'],
 ['0268057eb'

In [3]:
recording_id= data[:,0]

TypeError: list indices must be integers or slices, not tuple

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
a=[[1,2,3],[4,5,6]]
print("取一行："+ str(a[0]))
print("取一列："+ str(a[:,0]))

取一行：[1, 2, 3]


TypeError: list indices must be integers or slices, not tuple

## List 切片取一列：列表解析
因为列表可以存放不同类型的数据，因此列表中每个元素的大小可以相同，也可以不同，也就不支持一次性读取一列    
所以需要用列表解析的方法读取一列：

In [5]:
b=[x[0] for x in a]
print(b)

[1, 4]


In [6]:
recording_id= [x[0] for x in data]
print(recording_id)

['recording_id', '003bec244', '006ab765f', '007f87ba2', '0099c367b', '009b760e6', '00b404881', '00d442df7', '011f25080', '015113cad', '0151b7d20', '01b41f92b', '0201197ec', '0209f7ab2', '0268057eb', '0275e127d', '0295e3234', '0297d886e', '02b9a8ab9', '0313e82cf', '03b96f209', '03b96f209', '03d77fede', '043356ff8', '04975ecd8', '04d6b1fc1', '04e70a8e3', '0509303a5', '050d0ca85', '053aeb7bd', '053aeb7bd', '055088446', '055d22517', '057802c25', '05b9c974c', '05f8c0f2f', '067f49f8b', '068f1b8e2', '06a0c48c9', '06c44d203', '073e4d908', '078788674', '07d1687e0', '07dcf40f7', '07f1227b0', '08db743d0', '090ae427d', '09315d9bf', '0968aea09', '09a946316', '0a350d11c', '0a9cdd8a5', '0ab6aa734', '0b2fa3f80', '0b3ef9c4e', '0c2124550', '0c448e77c', '0c48ed342', '0c936a1d4', '0cb4632d6', '0d25045a9', '0e034f968', '0e29d05ed', '0e2e4ac19', '0e799da44', '0ea8ea68a', '0eb2079ea', '0f8cadb4a', '0f99800b7', '0fc62dd3c', '100e4ddc9', '103db6411', '10dae79ed', '119b154cc', '11a52d37a', '11bafff5d', '11c2c02

In [7]:
unique_rec=np.unique(np.array(recording_id))
unique_rec.shape

(1133,)

一共1132条不同的音频，而data有1216条记录，是因为一条音频中出现了多种species，分别用data中的一行来记录该species在这条音频出现的时间段，频率。

In [8]:
type(data)

list

In [9]:
data1=np.array(data)
print(type(data1))
print(data1.shape)

<class 'numpy.ndarray'>
(1217, 7)


## array & list
* why is the data here as a list instead of an array?
* 因为data中的数据类型不同，data[0]第一行是每列的列名。
* list可以储存不同类型的数据，而array储存同种类型的数据。



In [10]:
# Check minimum/maximum frequencies for bird calls
# Not neccesary, but there are usually plenty of noise in low frequencies, and removing it helps
fmin = 24000
fmax = 0

# Skip header row (recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max) and start from 1 instead of 0
for i in range(1, len(data)):
    if fmin > float(data[i][4]):
        fmin = float(data[i][4])
    if fmax < float(data[i][6]):
        fmax = float(data[i][6])
# Get some safety margin
fmin = int(fmin * 0.9)
fmax = int(fmax * 1.1)
print('Minimum frequency: ' + str(fmin) + ', maximum frequency: ' + str(fmax))


Minimum frequency: 84, maximum frequency: 15056


In [11]:
print('Starting spectrogram generation')

for i in range(1, len(data)):
    # All sound files are 48000 bitrate, no need to slowly resample
    #librosa.load()读取音频
    #如果 sr 缺省，librosa会默认以22050的采样率读取音频文件，高于该采样率的音频文件会被下采样，低于该采样率的文件会被上采样。
    #sr=None, 以原始采样率读取音频文件。
    wav, sr = librosa.load('/kaggle/input/rfcx-species-audio-detection/train/' + data[i][0] + '.flac', sr=None)
    
    
    
    #tmin*sr 第几个采样点
    t_min = float(data[i][3]) * sr                          
    t_max = float(data[i][5]) * sr
    
    # Positioning sound slice
    center = np.round((t_min + t_max) / 2)
    beginning = center - length / 2
    if beginning < 0:
        beginning = 0
    
    ending = beginning + length
    if ending > len(wav):
        ending = len(wav)
        beginning = ending - length
        
    slice = wav[int(beginning):int(ending)]
    
    
    # Mel spectrogram generation
    # Default settings were bad, parameters are adjusted to generate somewhat reasonable quality images
    # The better your images are, the better your neural net would perform
    # You can also use librosa.stft + librosa.amplitude_to_db instead
    mel_spec = librosa.feature.melspectrogram(slice, n_fft=fft, hop_length=hop, sr=sr, fmin=fmin, fmax=fmax, power=1.5)
   
    

Starting spectrogram generation


In [12]:
mel_spec

array([[0.11448789, 0.24621549, 0.21081041, ..., 0.25604618, 0.17219068,
        0.37962174],
       [0.12642445, 0.24760567, 0.11077897, ..., 0.52956104, 0.14786983,
        0.6199483 ],
       [0.19142732, 0.3243736 , 0.14949024, ..., 0.6549945 , 0.6719812 ,
        0.4690652 ],
       ...,
       [0.00156854, 0.0016682 , 0.0028125 , ..., 0.00174236, 0.00251896,
        0.00310167],
       [0.00214433, 0.00254561, 0.00241195, ..., 0.00244218, 0.00190946,
        0.00293769],
       [0.00216653, 0.00226011, 0.00195679, ..., 0.00260401, 0.00222476,
        0.00195581]], dtype=float32)

In [13]:
print("len(data):"+str(len(data)))
print("wav.shape:"+str(wav.shape))
print(wav)
print("slice.shape:"+ str(slice.shape))
print(slice)

len(data):1217
wav.shape:(2880000,)
[ 0.04241943  0.04104614  0.04455566 ... -0.01660156 -0.01754761
 -0.01660156]
slice.shape:(480000,)
[-0.00372314 -0.00210571 -0.00549316 ...  0.01501465 -0.0027771
 -0.01556396]


wav是什么，是对train_tp里1216条记录，以原始采样率（sr=48KHZ?)读取每一条音频, 我以为会是：
    - 列数-每条音频的时间t*采样率sr
    - 行数-1216
    - wav.shape==(1216,t*48000)
但结果竟然wav.shape:(2880000,)，一维？ 2880000/48000=600，2880000和sr(48KHZ)有关系吗？

啊，太蠢了。在for循环里，wav只显示最后一条，后面的把前面的覆盖掉了呀。2880000/48000=60s,每条音频是一分钟嘛。


***


### librosa.feature.melspectrogram(y=None, sr=22050, S=None, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='reflect', power=2.0, **kwargs)[source]
  Compute a mel-scaled spectrogram.
- If a spectrogram input S is provided, then it is mapped directly onto the mel basis by mel_f.dot(S).

- If a time-series input y, sr is provided, then its magnitude spectrogram S is first computed, and then mapped onto the mel scale by mel_f.dot(S**power).
> slince是截取species从t_min到t_max这段时间的wav片段，所以slice和wav的格式应该是一样的。
> librosa.feature.melspectrogram(slice,...), input is a time-series。

In [14]:


    
    # Mel spectrogram generation
    # Default settings were bad, parameters are adjusted to generate somewhat reasonable quality images
    # The better your images are, the better your neural net would perform
    # You can also use librosa.stft + librosa.amplitude_to_db instead
    mel_spec = librosa.feature.melspectrogram(slice, n_fft=fft, hop_length=hop, sr=sr, fmin=fmin, fmax=fmax, power=1.5)


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session